# Final exam
Goal of this is to see if you can code up your own synthetic data generator and IFP

Use the following system definition

In [1]:
import numpy as np
from scipy.constants import c as speed_of_light
# Geometry 
arp_poly=np.array([
    [4800897.883632624, 57.036013951381584], [2752862.8001471036, -4.058634127391252], [3180199.1684093135, -82.03914066808467]
])
p_scp=np.array([4787610.688267582, 2764128.319646417,  3170373.7353836414])

num_pulses = 2000 # number of pulses we transmit
prf = 4000 # pulse repition frequency in Hz
bandwidth = 100e6 # trasnmit RF bandwidth in Hz
tx_pulse_len = 1e-05 # Duration of the pulse we send in seconds
t_start = 0.0 # time when we start transmitting pulses in seconds
t_end = 0.4998734257422694 # time of last pulse we sent in seconds
chirp_rate = bandwidth / tx_pulse_len # chirp rate in Hz/s
fc = 10e9 # center frequency of the system in Hz
wavelen = speed_of_light / fc # wavelenght of the center frequecny in meters
rx_window_factor = 3 # multiplier on the pulse length for how long we listen
rx_nyquist_factor = 1.25 # how much over nyquist rate we oversample
sampling_rate = 2 * bandwidth * rx_nyquist_factor # ADC sampling rate in Hz 
sampling_period = 1 / sampling_rate # ADC sampling period in seconds
rx_window_duration = rx_window_factor * tx_pulse_len # Duration of the receive window in seconds
tx_times = np.arange(0, t_end, 1/prf) # Array of times we send pulses in seconds
t_adc = np.arange(0, rx_window_duration, sampling_period) # ADC sampling times in seconds
t_adc -= t_adc.mean()
num_samples = t_adc.size

# Create Scene 
To create an image we need to define some points to image.  To do this we will define points in the slant plane and convert to them to ECEF.  Recall the defition of the slant plane is
The definition of the slant plane coordinate system is:
$$
\text{look} = \begin{cases}
1 & (\mathbf{u}_{p_{coa}} \times \mathbf{u}_{v_{coa}}) \cdot \mathbf{u}_{los},\\
0  & 0 \text{ else}
\end{cases}
$$  

$$\mathbf{z}_{sp} = \text{look}\cdot (\mathbf{u}_{v_{coa}} \times \mathbf{u}_{los})$$
$$\mathbf{x}_{sp} = -\mathbf{u}_{los}$$
$$\mathbf{y}_{sp} = \mathbf{z}_{sp} \times \mathbf{x}_{sp}$$
$$\mathbf{R}_{\text{sp}\rightarrow\text{ecef}} = [\mathbf{x}_{sp}, \mathbf{y}_{sp}, \mathbf{z}_{sp}]$$
To convert an ecef position to slant plane position compute:
$$\mathbf{p}_{\text{sp}} = \mathbf{R}_{\text{sp}\rightarrow\text{ecef}}(\mathbf{p}_{\text{ecef}} - \mathbf{p}_{\text{scp}})$$


## Problem 1
Write a function to compute $\mathbf{R}_{\text{sp}\rightarrow\text{ecef}}$

## Solution

In [ ]:
# Solution
import numpy as np
import numpy.polynomial.polynomial as npp
def slant_plane_to_ecef(p_arp_ecef, v_arp_ecef, p_grp_ecef):
    x = p_arp_ecef - p_grp_ecef
    slt_rg = np.linalg.norm(x, axis=-1)
    u_x = x / slt_rg
    z = np.cross(u_x, v_arp_ecef, axisa=-1, axisb=-1)

    look = 1
    u_z = np.expand_dims(look, axis=-1) * (z / (np.linalg.norm(z, axis=-1)))
    u_y = np.cross(u_z, u_x, axisa=-1, axisb=-1)
    slant_to_ecef = np.stack((u_x, u_y, u_z), axis=-1)
    return slant_to_ecef

# Test the function for the center of aperture (COA) time
coa_time = (t_end - t_start) / 2 
p_coa = npp.polyval(coa_time, arp_poly.T).T
v_coa = npp.polyval(coa_time, npp.polyder(arp_poly.T)).T
sp_2_ecef = slant_plane_to_ecef(p_coa, v_coa, p_scp)

## Problem 2
Define points in the slant plane coordinate system and convert them to ECEF.  Write code which implements the following equation
$$\mathbf{p}_{\text{sp}} = \mathbf{R}_{\text{sp}\rightarrow\text{ecef}}(\mathbf{p}_{\text{ecef}} - \mathbf{p}_{\text{scp}})$$ 

Verify your code by plotting the points in the slant plane coordinate system.


## Solution

In [ ]:
# Solution
import matplotlib.pyplot as plt
import numpy as np
points_sp = np.stack([
    np.array([-1000, -500, 0]),
    np.array([-400, 400, 0]),
    np.array([0, 0, 0]),
    np.array([300, 300, 0]),
    np.array([600, -600, 0]),
])
rotation = sp_2_ecef @ points_sp.T
points_ecef = rotation.T + p_scp


plt.figure()
plt.plot(points_sp[:, 0], points_sp[:, 1], 'o')
plt.xlabel('Slant plane X (meters)')
plt.ylabel('Slant plane Y (meters)')
plt.title('Target locations in slant plane')
plt.show()

# Create Waveform
Recall that the definition of a baseband linear frequency modulated waveform (LFM) is defined by:
$$x(t) = e^{j\pi\gamma t^2}$$
where $\gamma$ is the chirp rate.  

## Problem 1
Write a function to compute the response of a LFM.

## Solution

In [ ]:
# Solution
import numpy as np
def lfm(t, pulse_duration):
    phase = np.pi * chirp_rate * t * t
    half_dur = 0.5 * pulse_duration
    t_max, t_min = half_dur, -half_dur

    mag = np.logical_and(
        np.less_equal(t, t_max),
        np.greater_equal(t, t_min)
    ).astype(float)

    return mag * (np.cos(phase) + 1j * np.sin(phase))

## Problem 2
Recall that the waveform response to a point scattereer is defined by:  

$$x(t) = e^{\pi \gamma (t-\frac{2R}{c})^2}$$

Where $R$ is the distance to each point scatterer.

Using the monostatic approximation $R_{tx}$ = $R_{rx}$, for the center of aperture time, compute what the response would look like for a single pulse to all the points in the scene.  Verify your code by plotting the magitude ($|x|^2$) of the return from each point scatterer individually to show where each pulse lies in the receive window.  Points that have a +x position in the slant plane should be closer to the start of the receive window and points that have a -x position in the slant plane should be closer to the end of the receive window

## Solution

In [ ]:
# solution
coa_pulse = num_pulses // 2
targets = [points_ecef[ii,:].flatten() for ii in range(points_ecef.shape[0])]
arp_pos = npp.polyval(tx_times[i_pulse], arp_poly.T).T

pulse_return_per_target = np.zeros((len(targets), t_fast.size), dtype=np.complex128)
for ii, tgt_pos enumerate(targets):
    monostatic_distance = 2 * np.linalg.norm(tgt_pos - arp_pos)
    
    # Using the monostatic approximation we will receive the pulse rx_time
    # past tx_time
    rx_time = monostatic_distance / speed_of_light
    
    # Center the ADC around the time delay to the target.  We will clip this to the receive window
    # in the lfm function. i,e) if a time delay is outside of the receive window the response will
    # be all 0s
    t_fast = t_adc - rx_time
    
    # Compute fast time term.  
    pulse_return_per_target[ii, :] = lfm(t_fast)

# SAR signal simulation